# House Prices - EDA and models comparison

*Autores: David Tejero Ruiz & Miguel Gil Castilla*

El [dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) ha sido obtenido de la página web de Kaggle en el apartado de competiciones. Este ha sido escogido debido a que está recomendado como un dataset de "juguete" con el que continuar el aprendizaje en python y ciencia del dato a un nivel más o menos básico. Nuestra idea es realizar un exploratory data análisis aplicando técnicas vistas en clase y ampliando estas con algunas ideas captadas de la red, tras esto trataremos nuestro dataset ya modificado para crear un modelo de regresión con el que predecir la variable respuesta *PriceSale*.

Una particularidad de este dataset es su elevado número de atributos (79), lo que lo hace muy adecuado para poner en práctica algunas técnicas vistas de ingeniería de características. Así, podemos extraer información relevante de ellas para realizar una predicción más o menos precisa del precio de venta de las diferentes viviendas.


Dado que en el conjunto test.csv no se proporciona el valor de la variable respuesta (era objetivo obtenerlo para la competición de Kaggle al que pertenece), nos hemos centrado en el conjunto *train.csv*, que hemos renombrado al archivo *data.csv* que se encuentra en este directorio, y será el conjunto de datos que analicemos.

In [ ]:
# Importamos el conjunto de datos
import pandas as pd
House_prices = pd.read_csv('data.csv', keep_default_na=False, na_values=[""])

# Para evitar avisos innecesarios (FutureWarnings):
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

Notar que Pandas por defecto, trata los valores NA (Not Available) como NaN (Not a Number). Si se analiza la descripción de los datos, NA es un valor que puede tomar el atributo, por lo que no se trata de un valor perdido como tal, sino que  indica que no se puede calcular el valor del atributo, porque la vivienda no dispone de el item que se está evaluando. Por ejemplo, en el caso del atributo de calidad de la piscina, NA indica que la vivienda no dispone de piscina, lo cuál es información relevante, y no un valor perdido que tendríamos que imputar.

In [ ]:
# Vemos la cabecera de los datos
House_prices.head()

Como vemos, el dataset está compuesto por 81 columnas, de las cuales 79 serán las características que usaremos en la predicción, y las dos restantes sol el id (que no aporta información), y la variable que queremos predecir (precio de venta de la casa).

Además notamos que existe una riqueza de características alta, encontrando tanto valores numéricos como categóricos, cosa que analizaremos a continuación en el análisis.

# Preprocesado de datos

In [ ]:
# Vamos a definir un problema de regresión sobre el precio de las casas, por lo que la variable 
# objetivo del dataset será 'SalePrice' (última columna de nuestro dataset como es habitual)
y_data = House_prices['SalePrice']

# Para tratar los diferentes atributos lo primero es distinguir entre las variables cuantitativas 
# (numéricas) y cualitativas (categóricas)

# Ya que el tipo de dato 'object' engloba a las variables categóricas y a las variables de tipo
# string, se puede usar dicho tipo de dato para distinguir las variables en numéricas y categóricas
numerical_atr = [col for col in House_prices.columns if House_prices.dtypes[col] != 'object']
categorical_atr = [col for col in House_prices.columns if House_prices.dtypes[col] == 'object']

# Eliminamos las variables 'SalePrice' (variable objetico) e 'Id' (no aporta información) de la
# lista de variables numéricas
numerical_atr.remove('SalePrice')
numerical_atr.remove('Id')

Los valores perdidos sí se encuentran representados por NaNs en nuestro pandas dataframe, que serán aquellos para los que falte un valor para la característica deseada.

Estos valores sí serían susceptibles de realizar una imputación con algunos de los métodos vistos (media, mediana, modelo predictivo, etc.) Sin embargo, nuestro dataset no contiene ninguno de estos valores

In [ ]:
# Vemos si existen algun valor nulo
print("Hay ", House_prices.isnull().sum().sum(), " valores nulos")

In [ ]:
print("Hay ", len(numerical_atr), " datos numéricos: ",numerical_atr)
print("Hay ", len(categorical_atr), " datos categóricos: ",categorical_atr)

Por otro lado, como comentamos al inicio, el dataset sí que contiene valores NA, que realmente podrían aportar información al decir que no se tiene el objeto que tratamos de evaluar en el atributo. Vamos a ver qué atributos contienen valores NA, y cuántos de estos valores NA contienen.

In [ ]:
# Vamos a ver que atributos contienen valores NA
na_counts = House_prices.apply(lambda x: x.value_counts().get('NA', 0))
na_counts = na_counts[na_counts > 0]
print("Atributos que contienen NA: \n{}".format(na_counts))

In [ ]:
# Mostramos a continuación una gráfica la información anterior
import matplotlib.pyplot as plt

ordenados_na = na_counts.sort_values(ascending=False)
plt.bar(ordenados_na.index, ordenados_na)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# Pitnamos histograma de la variable objetivo
plt.hist(y_data, bins=50)
plt.show()

In [ ]:
# Extraemos estadísticas de la variable respuesta (SalePrice)
y_data.describe()


In [ ]:
# Mostramos los valores de PoolQC respecto al precio de la casa
# plot = House_prices.plot.scatter(x='SalePrice', y=ordenados_na.index[0])
# plt.show()

fig, ax = plt.subplots(2, 2, figsize=(15, 7))
ax[0,0].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[0]])
ax[0,0].set_xlabel('SalePrice')
ax[0,0].set_ylabel(ordenados_na.index[0])
ax[0,0].grid()

ax[0,1].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[1]])
ax[0,1].set_xlabel('SalePrice')
ax[0,1].set_ylabel(ordenados_na.index[1])
ax[0,1].grid()

ax[1,0].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[2]])
ax[1,0].set_xlabel('SalePrice')
ax[1,0].set_ylabel(ordenados_na.index[2])
ax[1,0].grid()

ax[1,1].scatter(House_prices['SalePrice'], House_prices[ordenados_na.index[3]])
ax[1,1].set_xlabel('SalePrice')
ax[1,1].set_ylabel(ordenados_na.index[3])
ax[1,1].grid()

plt.show()

Como se puede observar en las gráficas comparativas, vemos que los valores que no son NA de los atributos categóricos se mantienen cerca de la media de la variable respuesta con lo cual no aportan información relevante. Es por esto por lo que decidimos directamente eliminar estos atributos categóricos.

In [ ]:
# Eliminamos los 4 atributos que más NA contienen 
House_prices = House_prices.drop(ordenados_na.index[:4], axis=1)
# Eliminamos también los Id que no nos aportarán información de valor predicivo
House_prices = House_prices.drop('Id', axis=1)

## Multicolinealidad

In [ ]:
# Coeficiente de correlación de Pearson entre los atributos
import seaborn as sns
plt.figure(figsize=(15,15))
plt.title('Coeficiente de Correlación de Pearson entre los atributos', y=1.05, size=15)

sns.heatmap(House_prices.corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap='viridis', linecolor='white', annot=True)

plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

In [ ]:
# Una vez que hemos acabado nuestra fase inicial de selección de características, vamos a construir
# nuestro dataset final

# Dado que las variables categóricas son tratadas de forma diferente a las variables numéricas,
# ya que hay que transformarlas en vaiables numéricas, utilizamos dos dataframes diferentes para
# almacenar las variables numéricas y las variables categóricas
numerical_data = House_prices[numerical_atr]

# Para tratar las variables categóricas vamos a usar el método de 'one-hot encoding' que consiste
# en crear una nueva variable binaria por cada valor que pueda tomar la variable categórica.
# Para ello, usamos la función 'get_dummies' de pandas que efectúa dicho 'one-hot encoding' de
# forma automática
categorical_data = pd.get_dummies(House_prices[categorical_atr])

# Concatenamos las variables numéricas y las variables categóricas para obtener el dataset final
X_data = pd.concat([numerical_data, categorical_data], axis=1)

# Modelos

In [ ]:
# Dividimos los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=0)



COSAS QUE HAY QUE HACER:

**Preprocesado de datos**
- Eliminar columnas con muchos valores perdidos
- One-hot encoding / Label encoding (categorical variables)
- Normalizar datos: StandardScaler, MinMaxScaler, RobustScaler 
- Eliminar outliers: IQR, Z-score, etc.  (?)
- Discretizado de variables continuas (?): Binning, etc.


**Ingeneering features**
- Reducción de dimensionalidad: *PCA*, LDA, etc.
- Selección de variables - Filter Methods: Correlation, Chi2, ANOVA, etc.
- Multicolinealidad: VIF, etc. 
- Crear nuevas variables.

**Posibles Modelos**
- Regresión lineal
- Regresión polinómica
- Regresión Ridge
- Regresión Lasso
- Regresión ElasticNet
- SVR
- Random Forest
- **Gradient Boosting**
- XGBoost
- LightGBM
- CatBoost

**Evaluación de modelos**
- MSE
- k-fold cross validation
- Recall, Precision, F1-score, etc. (?)